In [4]:
import warnings, logging
from skmultiflow.meta import LeveragingBaggingClassifier
from skmultiflow.lazy import KNNClassifier
from skmultiflow.data import SEAGenerator
from timeit import default_timer as timer
%matplotlib notebook

    This demo tests the LeveragingBaggingClassifier on a file stream, which gives
    instances coming from a SEA generator. 

    The test computes the performance of the LeveragingBaggingClassifier as well
    as the time to create the structure and classify max_samples (2000 by default) 
    instances.

In [5]:
start = timer()

logging.basicConfig(format='%(message)s', level=logging.INFO)
warnings.filterwarnings("ignore", ".*Passing 1d.*")

# Setup stream
stream = SEAGenerator(1, noise_percentage=0.067, random_state=1)

# Setup classifier
clf = LeveragingBaggingClassifier(base_estimator=KNNClassifier(n_neighbors=8, max_window_size=2000, leaf_size=30),
                                    n_estimators=1, random_state=1)

sample_count = 0
correctly_classified = 0
max_samples = 2000
train_size = 200
first = True
if train_size > 0:
    X, y = stream.next_sample(train_size)
    clf.partial_fit(X, y, classes=stream.target_values)
    first = False

logging.info('%s%%', 0.0)
while sample_count < max_samples:
    if (sample_count+1) % (max_samples / 20) == 0:
        logging.info('%s%%', str(((sample_count // (max_samples / 20)+1) * 5)))
    X, y = stream.next_sample(2)
    my_pred = clf.predict(X)
    if first:
        clf.partial_fit(X, y, classes=stream.target_values)
        first = False
    else:
        clf.partial_fit(X, y)

    if my_pred is not None:
        if y[0] == my_pred[0]:
            correctly_classified += 1
    
    sample_count += 1

end = timer()

0.0%
5.0%
10.0%
15.0%
20.0%
25.0%
30.0%
35.0%
40.0%
45.0%
50.0%
55.0%
60.0%
65.0%
70.0%
75.0%
80.0%
85.0%
90.0%
95.0%
100.0%


In [7]:
print('Evaluation time: ' + str(end - start))
print(str(sample_count) + ' samples analyzed.')
print('My performance: ' + str(correctly_classified / sample_count))

Evaluation time: 9.2202634100031
2000 samples analyzed.
My performance: 0.843
